In [1]:
%pip install langchain_groq langchain_core langchain_community pypdf chromadb sentence_transformers gradio tf_keras ipywidgets vaderSentiment

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 3.7 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.5 MB 3.5 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 2.9 MB/s eta 0:00:01
   ------------------------------------- --

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


In [5]:
import os
import json
from collections import deque
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langchain_groq import ChatGroq
import datetime
import random

# File to store user data
USER_DATA_FILE = "user_data.json"

# University mental health resources
UNIVERSITY_RESOURCES = {
    "Centennial College": "Visit the Student Wellness Centre: https://www.centennialcollege.ca/student-health",
    "University of Toronto": "Check U of T’s mental health services: https://mentalhealth.utoronto.ca/",
}

# Motivational quotes for students
MOTIVATIONAL_QUOTES = [
    "Stay focused! Every small step brings you closer to success. 💪",
    "You’re capable of amazing things—keep pushing forward!",
    "Don’t let stress take over! Take breaks, breathe, and keep going. 🚀"
]

# Load user data from file
def load_user_data():
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, "r") as file:
            return json.load(file)
    return {}

# Save user data to file
def save_user_data(user_data):
    with open(USER_DATA_FILE, "w") as file:
        json.dump(user_data, file, indent=4)

# Update user data
def update_user_data(user_id, key, value):
    user_data = load_user_data()
    if user_id not in user_data:
        user_data[user_id] = {}
    user_data[user_id][key] = value
    save_user_data(user_data)

# Collect student-specific details
def update_student_profile(user_id):
    user_data = load_user_data()
    
    if user_id not in user_data:
        user_data[user_id] = {}

    # Ask for details if not already stored
    if "major" not in user_data[user_id]:
        user_data[user_id]["major"] = input("What’s your major? ")
    if "year_of_study" not in user_data[user_id]:
        user_data[user_id]["year_of_study"] = input("What year are you in? ")
    if "common_stressors" not in user_data[user_id]:
        user_data[user_id]["common_stressors"] = input("What are some things that stress you out? ")
    if "university" not in user_data[user_id]:
        user_data[user_id]["university"] = input("Which university are you studying at? ")

    save_user_data(user_data)
    print("Profile updated! I’ll remember this next time we chat.")

# Get mental health resources for a student's university
def get_mental_health_resources(user_id):
    user_data = load_user_data()
    university = user_data[user_id].get("university", "your university")
    
    if university in UNIVERSITY_RESOURCES:
        return f"If you need support, check out {UNIVERSITY_RESOURCES[university]}"
    else:
        return "I recommend checking your university's website for student wellness resources."

# Analyze sentiment of user input
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(text)
    compound_score = sentiment_scores["compound"]

    if compound_score <= -0.5:
        return "sad"
    elif -0.5 < compound_score <= -0.1:
        return "frustrated"
    elif -0.1 < compound_score < 0.1:
        return "neutral"
    elif 0.1 <= compound_score < 0.5:
        return "happy"
    else:
        return "excited"

# Load the LLM
def load_llm():
    return ChatGroq(
        temperature=0.2,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

# Generate a summary of the conversation
def generate_summary(conversation_history, llm):
    prompt = f"""
    You are a helpful assistant. Summarize this conversation in 1-2 sentences.

    Conversation:
    {" ".join(conversation_history)}

    Summary:
    """
    response = llm.invoke(prompt)
    return response.content.strip()

# Check deadlines & send reminders
def check_deadlines(user_id):
    user_data = load_user_data()
    deadlines = user_data[user_id].get("deadlines", {})
    
    today = datetime.date.today()
    upcoming = [task for task, date in deadlines.items() if datetime.date.fromisoformat(date) <= today + datetime.timedelta(days=3)]
    
    if upcoming:
        return f"Reminder! You have upcoming deadlines: {', '.join(upcoming)}. Don’t forget to plan ahead!"
    return "No major deadlines soon. Keep up the good work!"

# Send daily motivational messages
def send_daily_motivation():
    return random.choice(MOTIVATIONAL_QUOTES)

# Chatbot loop
def chatbot_loop():
    print("Starting mental health chatbot...")
    llm = load_llm()

    # Load user data
    user_data = load_user_data()

    # Ask for user details
    user_id = input("Chatbot: Hi! What's your name? ")
    if user_id not in user_data:
        print(f"Chatbot: Nice to meet you, {user_id}! Let's get started.")
        update_user_data(user_id, "name", user_id)
        update_student_profile(user_id)  # Ask for student-specific details
    else:
        print(f"Chatbot: Welcome back, {user_id}! How are you feeling today?")
        print(check_deadlines(user_id))  # Check for upcoming deadlines
        print("Chatbot:", send_daily_motivation())  # Send a motivational message

    conversation_history = deque(maxlen=50)

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            # Analyze overall mood
            analyzer = SentimentIntensityAnalyzer()
            user_messages = [msg for msg in conversation_history if msg.startswith("You:")]
            overall_sentiment = analyzer.polarity_scores(" ".join(user_messages))["compound"]

            # Map to mood
            mood = "sad" if overall_sentiment <= -0.5 else "frustrated" if -0.5 < overall_sentiment <= -0.1 else "neutral" if -0.1 < overall_sentiment < 0.1 else "happy" if 0.1 <= overall_sentiment < 0.5 else "excited"

            # Generate summary
            summary = generate_summary(conversation_history, llm)

            # Save user data
            update_user_data(user_id, "last_emotion", mood)
            update_user_data(user_id, "last_conversation", summary)

            # Provide a conclusion
            print(f"\nChatbot: It seems like you're feeling {mood}.")
            print(get_mental_health_resources(user_id))  # Show university mental health services

            print("Chatbot: Wishing you well! Take care!")
            break

        # Add user input to conversation history
        conversation_history.append(f"You: {user_input}")

        # Generate response based on student details
        user_data = load_user_data()
        major = user_data[user_id].get("major", "student")

        prompt = f"""
        You are a mental health assistant for students. The user is studying {major}.
        - Name: {user_id}
        - Last emotion: {user_data[user_id].get("last_emotion", None)}
        - Last conversation: {user_data[user_id].get("last_conversation", None)}

        User: {user_input}
        Assistant:
        """
        response = llm.invoke(prompt)
        print(f"Chatbot: {response.content}")

        # Add bot response to conversation history
        conversation_history.append(f"Chatbot: {response.content}")

# Run chatbot
if __name__ == "__main__":
    chatbot_loop()


Starting mental health chatbot...
Chatbot: Welcome back, Gabriel! How are you feeling today?
No major deadlines soon. Keep up the good work!
Chatbot: Stay focused! Every small step brings you closer to success. 💪
Chatbot: Hey Gabriel, sorry to hear that you're feeling stressed out. It's totally normal, especially with the demands of studying Computer Science. What's been going on that's causing you to feel this way? Is it a particular assignment, exam, or something else entirely? Let's chat about it and see if we can break it down together.
Chatbot: It sounds like you're feeling a bit overwhelmed with multiple things on your plate and an exam tomorrow. That can be really stressful. Can you tell me a bit more about what's going on and how you're feeling? What are the multiple things that are on your mind, and what's worrying you about the exam? Sometimes talking through your concerns can help clarify things and make them feel more manageable. I'm here to listen and help if I can.
Chatbo